In [1]:
%%writefile score.py
import json
import numpy as np
import os
import tensorflow as tf

from azureml.core.model import Model

def init():
    global X, output, sess
    tf.reset_default_graph()
    model_root = Model.get_model_path('crash-detection')
    saver = tf.train.import_meta_graph(os.path.join(model_root, 'final_model.meta'))
    X = tf.get_default_graph().get_tensor_by_name("network/X:0")
    output = tf.get_default_graph().get_tensor_by_name("network/output/MatMul:0")
    
    sess = tf.Session()
    saver.restore(sess, os.path.join(model_root, 'crash-detection'))

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    out = output.eval(session=sess, feed_dict={X: data})
    y_hat = np.argmax(out, axis=1)
    print("y_hat: ", y_hat)
    return y_hat.tolist()

Writing score.py


In [2]:
from azureml.core.runconfig import CondaDependencies

cd = CondaDependencies.create()
cd.add_conda_package('opencv')
cd.add_tensorflow_conda_package()
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

print(cd.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults==1.0.23.*
- opencv
- tensorflow=1.10.0



In [3]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={'name':'CrashDetecton', 'framework': 'TensorFlow DNN'},
                                               description='Tensorflow DNN for Accident Detection')

In [4]:
from azureml.core.image import ContainerImage

imgconfig = ContainerImage.image_configuration(execution_script="score.py", 
                                               runtime="python", 
                                               conda_file="myenv.yml")

In [ ]:
# use an image available in a private Container Registry
imgconfig.base_image = "crashdetector.azurecr.io/maskrcnn"
imgconfig.base_image_registry.address = "crashdetector.azurecr.io"
imgconfig.base_image_registry.username = "crashdetector"
imgconfig.base_image_registry.password = "Ycy+YnxoyvYruvORaPl1WqwxWbW7Buu6"

In [5]:
%%time
from azureml.core.webservice import Webservice
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

model=Model(ws, 'crash-detection')

# model.download(target_dir='/Users/Amrut/CrashDetection/CarCrashModel/', exist_ok=True)

# # verify the downloaded model file
# file_path = os.path.join('/Users/Amrut/CrashDetection/CarCrashModel/outputs', "final_model.meta")

os.stat(file_path)

service = Webservice.deploy_from_model(workspace=ws,
                                       name='crash-detection',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=imgconfig)

service.wait_for_deployment(show_output=True)

print(service.get_logs())
print(service.scoring_uri)

Found the config file in: /Users/Amrut/CrashDetection/config.json
Workspace name: AccidentDetection
Azure region: eastus
Subscription id: e298a653-a33f-4b32-9ec2-2adfbd3b649d
Resource group: CCGroup7Resources
Creating image
Image creation operation finished for image crash-detection:4, operation "Succeeded"
Creating service
Running.......................................................................................................................................................................................................................................
TimedOutACI service creation operation finished, operation "TimedOut"
Service creation polling reached terminal state, current service state: Unhealthy
{
  "code": "AciHealthEndpointNotUp",
  "message": "Health endpoint is not up"
}
2019-04-26T21:57:16,831249083+00:00 - iot-server/run 
2019-04-26T21:57:16,831919425+00:00 - rsyslog/run 
2019-04-26T21:57:16,831249583+00:00 - gunicorn/run 
2019-04-26T21:57:16,832743777+00:00 - nginx/r

In [6]:
print(service.scoring_uri)

http://52.190.26.205:80/score


In [9]:
service.update(auth_enabled = True)

In [10]:
primary, secondary = service.get_keys()
print(primary)

MSiXjkWWLhmtOFtyK0CzChJN9gn85EQZ
